### ACTUAL

In [ ]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

from statsmodels.tsa.holtwinters import ExponentialSmoothing



import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()


def train_holt_winters(train, test_y, country, indicator):
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"Holt_Winters_{country}.json")
    best_params = None

    # Check if the best parameters JSON file exists
    if os.path.exists(params_file):
        with open(params_file, "r") as f:
            best_params = json.load(f)
        print(f"Loaded best parameters for {country} - {indicator} from {params_file}: {best_params}")
    else:
        # If not, run the grid search
        print(f"No pre-existing parameters for {country} - {indicator}. Running grid search.")
        param_grid = {
            'trend': [None, 'add', 'mul'],  # Focus on trend options
            'damped': [True, False],       # Damped trend
            'smoothing_level': [None, 0.1, 0.3, 0.5, 0.7, 0.9],  # Smoothing for level
            'smoothing_slope': [None, 0.1, 0.3, 0.5, 0.7, 0.9],  # Smoothing for trend
            'initialization_method': [None],
        }

        best_rmse = float('inf')
        for trend in param_grid['trend']:
            for damped in param_grid['damped']:
                for alpha in param_grid['smoothing_level']:
                    for beta in param_grid['smoothing_slope']:
                        for init_level in param_grid['initialization_method']:
                            try:
                                # Initialize model
                                model = ExponentialSmoothing(
                                    train,
                                    trend=trend,
                                    damped_trend=damped,
                                    seasonal=None
                                )
                                # Fit model with specific smoothing parameters
                                fitted_model = model.fit(
                                    smoothing_level=alpha,
                                    smoothing_slope=beta,
                                    initial_level=init_level,
                                    optimized=True
                                )
                                # Forecast and calculate RMSE
                                predictions = fitted_model.forecast(len(test_y))
                                rmse = np.sqrt(mean_squared_error(test_y, predictions))

                                # Track the best parameters
                                if rmse < best_rmse:
                                    best_rmse = rmse
                                    best_params = {
                                        "trend": trend,
                                        "damped_trend": damped,
                                        "smoothing_level": alpha,
                                        "smoothing_slope": beta,
                                        "initialization_method": init_level
                                    }
                            except Exception as e:
                                # Log errors for debugging
                                print(f"Error with parameters: trend={trend}, damped={damped}, "
                                      f"alpha={alpha}, beta={beta}. Error: {e}")
                                continue

        # Save the best parameters to JSON for future use
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)
        print(f"Saved best parameters for {country} - {indicator} to {params_file}: {best_params}")

    # Train the final model using the best parameters (loaded or discovered)
    best_model = ExponentialSmoothing(
        train,
        trend=best_params["trend"],
        damped_trend=best_params["damped_trend"]
    )
    best_fitted_model = best_model.fit(
        smoothing_level=best_params["smoothing_level"],
        smoothing_slope=best_params["smoothing_slope"],
        optimized=True
    )
    predictions = best_fitted_model.forecast(len(test_y))

    # Call the save_plot function to save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="Holt_Winters")

    print(f"Best Parameters used for {country} - {indicator}: {best_params}")
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions




with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()

                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)


                
                model_errors_rmse[(country, indicator)] = {}

                model_errors_rmse[(country, indicator)]['Holt-Winters'] , es_pred = train_holt_winters(df_original.iloc[:train_size]['Value'], 
                                                                                             df_original.iloc[train_size:]['Value'],
                                                                                             country,indicator)


                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])




from datetime import datetime
model ="ExponentialSmoothing"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)


### TEST

In [6]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

from statsmodels.tsa.holtwinters import ExponentialSmoothing



import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()


def train_holt_winters(train, test_y, country, indicator):
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"Holt_Winters_{country}.json")
    best_params = None

    # Check if the best parameters JSON file exists
    if os.path.exists(params_file):
        with open(params_file, "r") as f:
            best_params = json.load(f)
        print(f"Loaded best parameters for {country} - {indicator} from {params_file}: {best_params}")
    else:
        # If not, run the grid search
        print(f"No pre-existing parameters for {country} - {indicator}. Running grid search.")
        param_grid = {
            'trend': [None, 'add', 'mul'],  # Focus on trend options
            'damped': [True, False],       # Damped trend
            'smoothing_level': [None, 0.1, 0.3, 0.5, 0.7, 0.9],  # Smoothing for level
            'smoothing_slope': [None, 0.1, 0.3, 0.5, 0.7, 0.9],  # Smoothing for trend
            'initialization_method': [None],
        }

        best_rmse = float('inf')
        for trend in param_grid['trend']:
            for damped in param_grid['damped']:
                for alpha in param_grid['smoothing_level']:
                    for beta in param_grid['smoothing_slope']:
                        for init_level in param_grid['initialization_method']:
                            try:
                                # Initialize model
                                model = ExponentialSmoothing(
                                    train,
                                    trend=trend,
                                    damped_trend=damped,
                                    seasonal=None
                                )
                                # Fit model with specific smoothing parameters
                                fitted_model = model.fit(
                                    smoothing_level=alpha,
                                    smoothing_slope=beta,
                                    initial_level=init_level,
                                    optimized=True
                                )
                                # Forecast and calculate RMSE
                                predictions = fitted_model.forecast(len(test_y))
                                rmse = np.sqrt(mean_squared_error(test_y, predictions))

                                # Track the best parameters
                                if rmse < best_rmse:
                                    best_rmse = rmse
                                    best_params = {
                                        "trend": trend,
                                        "damped_trend": damped,
                                        "smoothing_level": alpha,
                                        "smoothing_slope": beta,
                                        "initialization_method": init_level
                                    }
                            except Exception as e:
                                # Log errors for debugging
                                print(f"Error with parameters: trend={trend}, damped={damped}, "
                                      f"alpha={alpha}, beta={beta}. Error: {e}")
                                continue

        # Save the best parameters to JSON for future use
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)
        print(f"Saved best parameters for {country} - {indicator} to {params_file}: {best_params}")

    # Train the final model using the best parameters (loaded or discovered)
    best_model = ExponentialSmoothing(
        train,
        trend=best_params["trend"],
        damped_trend=best_params["damped_trend"]
    )
    best_fitted_model = best_model.fit(
        smoothing_level=best_params["smoothing_level"],
        smoothing_slope=best_params["smoothing_slope"],
        optimized=True
    )
    predictions = best_fitted_model.forecast(len(test_y))

    # Call the save_plot function to save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="Holt_Winters")

    print(f"Best Parameters used for {country} - {indicator}: {best_params}")
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions




with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()

                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)


                
                model_errors_rmse[(country, indicator)] = {}

                model_errors_rmse[(country, indicator)]['Holt-Winters'] , es_pred = train_holt_winters(df_original.iloc[:train_size]['Value'], 
                                                                                             df_original.iloc[train_size:]['Value'],
                                                                                             country,indicator)


                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])




from datetime import datetime
model ="ExponentialSmoothing"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)


Loaded best parameters for Czech Republic - GDP per Capita (USD) from ../best_params\GDP per Capita (USD)\Holt_Winters_Czech Republic.json: {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.5, 'smoothing_slope': 0.1, 'initialization_method': None}


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\admin\AppData\Local\Temp\ipykernel_7612\3673667026.py:138: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  best_fitted_model = best_model.fit(
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


Best Parameters used for Czech Republic - GDP per Capita (USD): {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.5, 'smoothing_slope': 0.1, 'initialization_method': None}
Loaded best parameters for Czech Republic - GDP (USD) from ../best_params\GDP (USD)\Holt_Winters_Czech Republic.json: {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.5, 'smoothing_slope': 0.1, 'initialization_method': None}


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\admin\AppData\Local\Temp\ipykernel_7612\3673667026.py:138: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  best_fitted_model = best_model.fit(
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


Best Parameters used for Czech Republic - GDP (USD): {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.5, 'smoothing_slope': 0.1, 'initialization_method': None}
Loaded best parameters for Czech Republic - Inflation (CPI) from ../best_params\Inflation (CPI)\Holt_Winters_Czech Republic.json: {'trend': 'mul', 'damped_trend': True, 'smoothing_level': 0.9, 'smoothing_slope': 0.9, 'initialization_method': None}


...

Best Parameters used for Italy - GDP growth (annual %): {'trend': 'add', 'damped_trend': True, 'smoothing_level': 0.5, 'smoothing_slope': 0.9, 'initialization_method': None}
Loaded best parameters for Italy - Imports of goods and services (% of GDP) from ../best_params\Imports of goods and services (% of GDP)\Holt_Winters_Italy.json: {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.1, 'smoothing_slope': 0.1, 'initialization_method': None}


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\admin\AppData\Local\Temp\ipykernel_7612\3673667026.py:138: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  best_fitted_model = best_model.fit(
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


Best Parameters used for Italy - Imports of goods and services (% of GDP): {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.1, 'smoothing_slope': 0.1, 'initialization_method': None}
Loaded best parameters for Italy - Exports of goods and services (% of GDP) from ../best_params\Exports of goods and services (% of GDP)\Holt_Winters_Italy.json: {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.7, 'smoothing_slope': 0.1, 'initialization_method': None}


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\admin\AppData\Local\Temp\ipykernel_7612\3673667026.py:138: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  best_fitted_model = best_model.fit(


Best Parameters used for Italy - Exports of goods and services (% of GDP): {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.7, 'smoothing_slope': 0.1, 'initialization_method': None}


### OLD VERSIONS

In [ ]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

from statsmodels.tsa.holtwinters import ExponentialSmoothing



import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()


def train_holt_winters(train, test_y, country, indicator):
    param_grid = {
        'trend': [None, 'add', 'mul'],  # Focus on trend options
        'damped': [True, False],       # Damped trend
        'smoothing_level': [None, 0.1, 0.3, 0.5, 0.7, 0.9],  # Smoothing for level
        'smoothing_slope': [None, 0.1, 0.3, 0.5, 0.7, 0.9],  # Smoothing for trend
        'initialization_method': [None, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    }

    best_rmse = float('inf')
    best_params = None

    for trend in param_grid['trend']:
        for damped in param_grid['damped']:
            for alpha in param_grid['smoothing_level']:
                for beta in param_grid['smoothing_slope']:
                    for init_level in param_grid['initialization_method']:
                        try:
                            # Initialize model
                            model = ExponentialSmoothing(
                                train,
                                trend=trend,
                                damped_trend=damped,
                                seasonal=None
                            )
                            # Fit model with specific smoothing parameters
                            fitted_model = model.fit(
                                smoothing_level=alpha,
                                smoothing_slope=beta,
                                initial_level=init_level,
                                optimized=True
                            )
                            # Forecast and calculate RMSE
                            predictions = fitted_model.forecast(len(test_y))
                            rmse = np.sqrt(mean_squared_error(test_y, predictions))

                            # Track the best parameters
                            if rmse < best_rmse:
                                best_rmse = rmse
                                best_params = {
                                    "trend": trend,
                                    "damped_trend": damped,
                                    "smoothing_level": alpha,
                                    "smoothing_slope": beta,
                                    "initialization_method": init_level
                                }
                        except Exception as e:
                            # Log errors for debugging
                            print(f"Error with parameters: trend={trend}, damped={damped}, "
                                  f"alpha={alpha}, beta={beta}. Error: {e}")
                            continue

    # Train the final best model
    best_model = ExponentialSmoothing(
        train,
        trend=best_params["trend"],
        damped_trend=best_params["damped_trend"]
    )
    best_fitted_model = best_model.fit(
        smoothing_level=best_params["smoothing_level"],
        smoothing_slope=best_params["smoothing_slope"],
        optimized=True
    )
    predictions = best_fitted_model.forecast(len(test_y))

    # Save the best parameters to JSON
    params_dir = os.path.join("../best_params", indicator)
    os.makedirs(params_dir, exist_ok=True)
    params_file = os.path.join(params_dir, f"Holt_Winters_{country}.json")
    with open(params_file, "w") as f:
        json.dump(best_params, f, indent=4)

    # Call the save_plot function to save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="Holt_Winters")

    print(f"Best Parameters saved for {country} - {indicator}: {best_params}")
    return best_rmse, predictions



with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()

                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)


                
                model_errors_rmse[(country, indicator)] = {}

                model_errors_rmse[(country, indicator)]['Holt-Winters'] , es_pred = train_holt_winters(df_original.iloc[:train_size]['Value'], 
                                                                                             df_original.iloc[train_size:]['Value'],
                                                                                             country,indicator)


                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])




from datetime import datetime
model ="ExponentialSmoothing"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)
